In [4]:
#Profit #P=50*xw+60*xh+65*xe
        #Fabrication
        #2*xw+2*xh+3*xe<=700
        #Assembly
        #2*xw+3*xh+1*xe<=500
        
        #AwesomeBad
        #F3 A3 P60
        #Kickbutt
        #F5 A2 P85
        #FlamingSkull
        #F4 A4 P70
        #Dork (OCC)
        #F1 A1 P30
from scipy.optimize import linprog

#define values from the scenario
profit = [-60 , -85, -70, -30]
constraints = [[3 , 5 , 4, 1] , [3 , 2 , 4, 1]] 
allowance = [700,500]

#execute scipy implementation of Simplex Method
result = linprog(profit , constraints , allowance)
print(result.x)


[  0.          66.66666667   0.         366.66666667]


In [9]:
#Profit 
#P=50*xw+60*xh+65*xe
#Fabrication
#2*xw+2*xh+3*xe<=700
#Assembly
#2*xw+3*xh+1*xe<=500

from scipy.optimize import linprog 

# Define values from the scenario
profit = [-50, -60, -65]
constraints = [[2, 2, 3], [2, 3, 1]]
allowance = [700, 500]

# Execute scipy implementation of Simplex Method with 'highs' method
result = linprog(profit, A_ub=constraints, b_ub=allowance, method='highs') 
print(result.x)


[  0.         114.28571429 157.14285714]


In [18]:
import numpy as np

def gen_matrix(var, cons):
    tab = np.zeros((cons + 1, var + cons + 2))
    return tab

def next_round_r(table):
    m = min(table[:-1, -1])
    if m >= 0:
        return False
    else:
        return True

def next_round(table):
    lr = len(table[:, 0])
    m = min(table[lr - 1, :-1])
    if m >= 0:
        return False
    else:
        return True

def find_neg_r(table):
    lc = len(table[0, :])
    m = min(table[:-1, lc - 1])
    if m <= 0:
        n = np.where(table[:-1, lc - 1] == m)[0][0]
    else:
        n = None
    return n

def find_neg(table):
    lr = len(table[:, 0])
    m = min(table[lr - 1, :-1])
    if m <= 0:
        n = np.where(table[lr - 1, :-1] == m)[0][0]
    else:
        n = None
    return n

def loc_piv_r(table):
    total = []
    r = find_neg_r(table)
    row = table[r, :-1]
    m = min(row)
    c = np.where(row == m)[0][0]
    col = table[:-1, c]
    for i, b in zip(col, table[:-1, -1]):
        if i ** 2 > 0 and b / i > 0:
            total.append(b / i)
        else:
            total.append(10000)
    index = total.index(min(total))
    return [index, c]

def loc_piv(table):
    if next_round(table):
        total = []
        n = find_neg(table)
        for i, b in zip(table[:-1, n], table[:-1, -1]):
            if b / i > 0 and i ** 2 > 0:
                total.append(b / i)
            else:
                total.append(10000)
        index = total.index(min(total))
        return [index, n]

def pivot(row, col, table):
    lr = len(table[:, 0])
    lc = len(table[0, :])
    t = np.zeros((lr, lc))
    pr = table[row, :]
    if table[row, col] ** 2 > 0:
        e = 1 / table[row, col]
        r = pr * e
        for i in range(len(table[:, col])):
            k = table[i, :]
            c = table[i, col]
            if list(k) == list(pr):
                continue
            else:
                t[i, :] = list(k - r * c)
        t[row, :] = list(r)
        return t
    else:
        print('Cannot pivot on this element.')

def convert(eq):
    eq = eq.split(',')
    if 'G' in eq:
        g = eq.index('G')
        del eq[g]
        eq = [float(i) * -1 for i in eq]
        return eq
    if 'L' in eq:
        l = eq.index('L')
        del eq[l]
        eq = [float(i) for i in eq]
        return eq

def convert_min(table):
    table[-1, :-2] = [-1 * i for i in table[-1, :-2]]
    table[-1, -1] = -1 * table[-1, -1]
    return table

def gen_var(table):
    lc = len(table[0, :])
    lr = len(table[:, 0])
    var = lc - lr - 1
    v = []
    for i in range(var):
        v.append('x' + str(i + 1))
    return v

def add_cons(table):
    lr = len(table[:, 0])
    empty = []
    for i in range(lr):
        total = 0
        for j in table[i, :]:
            total += j ** 2
        if total == 0:
            empty.append(total)
    if len(empty) > 1:
        return True
    else:
        return False

def constrain(table, eq):
    if add_cons(table):
        lc = len(table[0, :])
        lr = len(table[:, 0])
        var = lc - lr - 1
        j = 0
        while j < lr:
            row_check = table[j, :]
            total = 0
            for i in row_check:
                total += float(i ** 2)
            if total == 0:
                row = row_check
                break
            j += 1

        eq = convert(eq)
        i = 0
        while i < len(eq) - 1:
            row[i] = eq[i]
            i += 1
        row[-1] = eq[-1]
        row[var + j] = 1
    else:
        print('Cannot add another constraint.')

def add_obj(table):
    lr = len(table[:, 0])
    empty = []
    for i in range(lr):
        total = 0
        for j in table[i, :]:
            total += j ** 2
        if total == 0:
            empty.append(total)
    if len(empty) == 1:
        return True
    else:
        return False

def obj(table, eq):
    if add_obj(table):
        eq = [float(i) for i in eq.split(',')]
        lr = len(table[:, 0])
        row = table[lr - 1, :]
        i = 0
        while i < len(eq) - 1:
            row[i] = eq[i] * -1
            i += 1
        row[-2] = 1
        row[-1] = eq[-1]
    else:
        print('You must finish adding constraints before the objective function can be defined.')

def maxz(table):
    while next_round_r(table) == True:
        table = pivot(loc_piv_r(table)[0], loc_piv_r(table)[1], table)
    lc = len(table[0, :])
    lr = len(table[:, 0])
    var = lc - lr - 1
    i = 0
    val = {}
    for i in range(var):
        col = table[:, i]
        s = sum(col)
        m = max(col)
        if float(s) == float(m):
            loc = np.where(col == m)[0][0]
            val[gen_var(table)[i]] = table[loc, -1]
        else:
            val[gen_var(table)[i]] = 0
    val['max'] = table[-1, -1]
    return val

def minz(table):
    table = convert_min(table)
    while next_round_r(table) == True:
        table = pivot(loc_piv_r(table)[0], loc_piv_r(table)[1], table)
    while next_round(table) == True:
        table = pivot(loc_piv(table)[0], loc_piv(table)[1], table)
    lc = len(table[0, :])
    lr = len(table[:, 0])
    var = lc - lr - 1
    i = 0
    val = {}
    for i in range(var):
        col = table[:, i]
        s = sum(col)
        m = max(col)
        if float(s) == float(m):
            loc = np.where(col == m)[0][0]
            val[gen_var(table)[i]] = table[loc, -1]
        else:
            val[gen_var(table)[i]] = 0
    val['min'] = table[-1, -1] * -1
    return val


In [19]:
m = gen_matrix(3,2)
constrain(m,'2,2,3,L,700')
constrain(m,'2,3,1,L,500')
obj(m,'52,60,65,0')
print(m)

print(maxz(m))
#m = gen_matrix(2,4) 
#constrain(m,'2,5,G,30') 
#constrain(m,'-3,5,G,5') 
#constrain(m,'8,3,L,85') 
#constrain(m,'-9,7,L,42') 
#obj(m,'2,7,0') 
#print(minz(m))

[[  2.   2.   3.   1.   0.   0. 700.]
 [  2.   3.   1.   0.   1.   0. 500.]
 [-52. -60. -65.   0.   0.   1.   0.]]
{'x1': 0, 'x2': 0, 'x3': 0, 'max': 0.0}


In [21]:
from scipy.optimize import linear_sum_assignment 
mins = [[22 , 30 , 10, 51,14],
[16 , 32 , 41, 64,57],
[13 , 39 , 78, 34,78],
[17 , 3 , 79, 17,53],
[44 , 73 , 9, 44,73]]
drones , cust = linear_sum_assignment(mins) 
print(drones+1) #drone IDs [ 1 , 2 , 3 ] 
print(cust+1) #customer IDs [ 2 , 1 , 3 ]


[1 2 3 4 5]
[5 1 4 2 3]


In [22]:
from scipy.optimize import linear_sum_assignment 
mins = [[32 , 27 , 19],
[ 25 , 48 , 34 ] ,
[ 26 , 44 , 30 ] ]
drones , cust = linear_sum_assignment(mins) 
print(drones+1) #drone IDs [ 1 , 2 , 3 ] 
print(cust+1) #customer IDs [ 2 , 1 , 3 ]

[1 2 3]
[2 1 3]


In [24]:
import numpy as np

def min_zero_row(zero_mat, mark_zero):
    '''
    The function can be split into two steps:
    #1 The function is used to find the row which containing the fewest 0.
    #2 Select the zero number on the row, and then mark the element corresponding row
    '''
    # Find the row
    min_row = [99999, -1]
    for row_num in range(zero_mat.shape[0]):
        if np.sum(zero_mat[row_num] == True) > 0 and min_row[0] > np.sum(zero_mat[row_num] == True):
            min_row = [np.sum(zero_mat[row_num] == True), row_num]
    # Mark the specific row and column as False
    zero_index = np.where(zero_mat[min_row[1]] == True)[0][0]
    mark_zero.append((min_row[1], zero_index))
    zero_mat[min_row[1], :] = False
    zero_mat[:, zero_index] = False

def mark_matrix(mat):
    '''
    Finding the returning possible solutions for LAP problem.
    '''
    # Transform the matrix to a boolean matrix (0 = True, others = False)
    cur_mat = mat
    zero_bool_mat = (cur_mat == 0)
    zero_bool_mat_copy = zero_bool_mat.copy()
    # Recording possible answer positions by marked_zero
    marked_zero = []
    while (True in zero_bool_mat_copy):
        min_zero_row(zero_bool_mat_copy, marked_zero)
    # Recording the row and column positions separately.
    marked_zero_row = []
    marked_zero_col = []
    for i in range(len(marked_zero)):
        marked_zero_row.append(marked_zero[i][0])
        marked_zero_col.append(marked_zero[i][1])
    # Step 2-2-1
    non_marked_row = list(set(range(cur_mat.shape[0])) - set(marked_zero_row))
    marked_cols = []
    check_switch = True
    while check_switch:
        check_switch = False
        for i in range(len(non_marked_row)):
            row_array = zero_bool_mat[non_marked_row[i], :]
            for j in range(row_array.shape[0]):
                # Step 2-2-2
                if row_array[j] == True and j not in marked_cols:
                    # Step 2-2-3
                    marked_cols.append(j)
                    check_switch = True
        for row_num, col_num in marked_zero:
            # Step 2-2-4
            if row_num not in non_marked_row and col_num in marked_cols:
                # Step 2-2-5
                non_marked_row.append(row_num)
                check_switch = True
    # Step 2-2-6
    marked_rows = list(set(range(mat.shape[0])) - set(non_marked_row))
    return (marked_zero, marked_rows, marked_cols)

def adjust_matrix(mat, cover_rows, cover_cols):
    cur_mat = mat
    non_zero_element = []
    # Step 4-1
    for row in range(len(cur_mat)):
        if row not in cover_rows:
            for i in range(len(cur_mat[row])):
                if i not in cover_cols:
                    non_zero_element.append(cur_mat[row][i])
    min_num = min(non_zero_element)
    # Step 4-2
    for row in range(len(cur_mat)):
        if row not in cover_rows:
            for i in range(len(cur_mat[row])):
                if i not in cover_cols:
                    cur_mat[row, i] = cur_mat[row, i] - min_num
    # Step 4-3
    for row in range(len(cover_rows)):
        for col in range(len(cover_cols)):
            cur_mat[cover_rows[row], cover_cols[col]] = cur_mat[cover_rows[row], cover_cols[col]] + min_num
    return cur_mat

def hungarian_algorithm(mat):
    dim = mat.shape[0]
    cur_mat = mat
    # Step 1 - Every column and every row subtract its internal minimum
    for row_num in range(mat.shape[0]):
        cur_mat[row_num] = cur_mat[row_num] - np.min(cur_mat[row_num])
    for col_num in range(mat.shape[1]):
        cur_mat[:, col_num] = cur_mat[:, col_num] - np.min(cur_mat[:, col_num])
    zero_count = 0
    while zero_count < dim:
        # Step 2 & 3
        ans_pos, marked_rows, marked_cols = mark_matrix(cur_mat)
        zero_count = len(marked_rows) + len(marked_cols)
        if zero_count < dim:
            cur_mat = adjust_matrix(cur_mat, marked_rows, marked_cols)
    return ans_pos

def ans_calculation(mat, pos):
    total = 0
    ans_mat = np.zeros((mat.shape[0], mat.shape[1]))
    for i in range(len(pos)):
        total += mat[pos[i][0], pos[i][1]]
        ans_mat[pos[i][0], pos[i][1]] = mat[pos[i][0], pos[i][1]]
    return total, ans_mat

def main():
    '''Hungarian Algorithm:
    Finding the minimum value in a linear assignment problem.
    Therefore, we can find the minimum value set in the net matrix
    by using Hungarian Algorithm. In other words, the maximum value
    and elements set in the cost matrix are available.
    '''
    # The matrix you want to find the minimum sum
    cost_matrix = np.array([[32, 27, 19],
                            [25, 48, 34],
                            [26, 44, 30]])
    ans_pos = hungarian_algorithm(cost_matrix.copy())  # Get the element position.
    ans, ans_mat = ans_calculation(cost_matrix, ans_pos)  # Get the minimum or maximum val
    # Show the result
    print(f"Linear Assignment problem result: {ans:.0f}\n{ans_mat}")


main()


Linear Assignment problem result: 82
[[ 0. 27.  0.]
 [25.  0.  0.]
 [ 0.  0. 30.]]
